In [7]:
import pandas as pd
from openai import OpenAI



In [2]:
df = pd.read_parquet('../Input/joined.parquet')
print(df.shape)
df.head()

(3000000, 13)


,Id,Title,Price,User_id,profileName,score,time,summary,text,authors,publisher,publishedDate,categories
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.0,1999-10-23,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,[Julie Strain],None,1996,[Comics & Graphic Novels]
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,5.0,2004-09-21,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,5.0,2004-03-09,Essential for every personal and Public Library,"If people become the books they read and if ""t...",[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.0,2004-07-25,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.0,2005-02-10,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]


## find authors

In [34]:
df_authors = df.explode('authors')
grouped_authors_df = df_authors.groupby('authors').agg(
    unique=('Title', 'nunique'),
    count=('score', 'size'),
    mean_score=('score', 'mean'),
    median_score=('score', 'median'),
    variance_score=('score', 'var')
).reset_index()

grouped_authors_df[grouped_authors_df['count']>20].sort_values(by=['variance_score', 'variance_score','mean_score'], ascending=False).head()

,authors,unique,count,mean_score,median_score,variance_score
76215,Joseph Natale Schneiderman,1,23,3.130435,5.0,4.027668
74058,John Shea,3,29,2.827586,1.0,4.004926
111405,Paul Finebaum,4,22,2.681818,1.0,3.941558
124807,Robert W. Wegner,1,22,3.000000,3.0,3.904762
38725,Edward G. Nawy,2,35,3.228571,5.0,3.887395


In [28]:
def get_reviews_by_author(df, author_name):
    # Filter the dataframe for the given author
    author_reviews = df[df['authors'] == author_name]
    
    # Create a dictionary to store one review per score
    reviews_dict = {}
    
    for _, row in author_reviews.iterrows():
        score = row['score']
        if score not in reviews_dict:
            reviews_dict[score] = {
                'Title': row['Title'],
                'Text': row['text'],
            }
    
    return reviews_dict

client = OpenAI()
def analyze_author_reviews(author_name, reviews_dict):
    # Prepare the reviews text to be included in the message
    reviews_text = "\n".join(
        [f"Book Title: {review['Title']}\nReader Review: {review['Text']}\n"
         for review in reviews_dict.values()]
    )
    
    # Create the system message
    system_message = (
        f"You are an experienced editor tasked with providing a comprehensive summary of book reviews for the author {author_name}. Your summary will be used internally by the publishing team to evaluate whether or not to sign this author.",
        "Focus on capturing the key strengths and weaknesses highlighted by reviewers, ensuring the summary is insightful and relevant for decision-making by publishing professionals"
        
    )
    
    # Send the request to OpenAI
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  
            messages=[
                {"role": "system", "content": str(system_message)},
                {"role": "user", "content": reviews_text}
            ]
        )
        
        # Extract and return the response
        return response.choices[0].message.content
    except Exception as e:
        return e

def main(df, author_name):
    athors_data = get_reviews_by_author(df, author_name)
    analysis_result = analyze_author_reviews(author_name ,athors_data)
    print(analysis_result)


In [35]:
main(df_authors, 'Paul Finebaum')

### Summary of Book Reviews for Paul Finebaum

#### Key Strengths:
1. **Satirical Impact**: Reviewers identified a strong satirical element in Finebaum's writing, particularly in "I Hate Penn State: 303 Reasons Why You Should, Too." The satire is noted to resonate well with readers who have a nuanced understanding of the sports culture being critiqued.
2. **Engagement and Provocation**: The books seem effective in engaging readers and sparking strong emotional responses. This provocative nature may enhance the book’s visibility and discussions around it, potentially driving sales through controversy and word of mouth.
3. **Audience Targeting**: One review highlighted the success of Finebaum's book as a gift to those who would appreciate its humor or shared animosity towards the subject. This indicates a clear niche market that Finebaum successfully caters to.

#### Key Weaknesses:
1. **Polarizing Content**: Not all reviewers appreciated the content, with significant negative feedback n